In [ ]:
!pip install scapy
!pip uninstall pyopenssl -y
!pip install pyopenssl==23.0.0
!pip uninstall cryptography -y
!pip install cryptography==38.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.9 MB/s eta 0:00:00
Found existing installation: pyOpenSSL 24.2.1
Uninstalling pyOpenSSL-24.2.1:
  Successfully uninstalled pyOpenSSL-24.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
Found existing installation: cryptography 39.0.2
Uninstalling cryptography-39.0.2:
  Successfully uninstalled cryptography-39.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [ ]:
import cryptography
import numpy as np
from scapy.layers.tls.record import *
from scapy.layers.tls.record_tls13 import *
import csv
from scapy.all import *
import os
import json
import time
load_layer("tls")


In [ ]:

target_directory = "/content/drive/MyDrive/TLSHandshakesTraffic"
# Get the list of directories dataset
directories = [d for d in os.listdir(target_directory) if os.path.isdir(os.path.join(target_directory, d))]
print("List of devices traffic to pre-process:")
for directory in directories:
    print(directory)

def extract_certificate(pkt):
    start_index = pkt.find("certs     = [")
    if start_index < 0:
        return " " # we check if there is a cert inside before going forward
    end_index = pkt.find("\n", start_index)
    certs = pkt[start_index:end_index + 1]
    cert_start = certs.find("[")
    cert_end = certs.find("\n")
    cert=certs[cert_start + 1:cert_end - 1]
    return cert

def data_existance(pkt,data):
  if pkt.show(dump=True).find(data) < 0:
    return ""
  else:
    return pkt[data]

def parse_tls_data(packet):
    if packet.haslayer(Raw):
        raw_payload = packet[Raw].load
        if raw_payload.startswith(b'\x16\x03'):
            try:
                tls_payload = TLS(raw_payload)
                return tls_payload
            except Exception as e:
                print("Could not parse TLS data:", e)
    return packet

List of devices traffic to pre-process:
18_74_2e_41_4d_35
20_df_b9_5f_41_7e
20_f8_5e_cc_18_1f
0_71_47_c0_91_93
0_e_f3_3b_85_e5
0_21_cc_4d_ce_8c
18_b4_30_c8_d8_28
0_17_88_68_5f_61
14_91_82_b4_4b_5f
0_3_7f_4f_c6_b5
68_db_f5_90_19_64
34_29_8f_1e_41_25
7c_c7_9_56_6e_48
50_c7_bf_5a_2e_a0
50_c7_bf_a0_f3_76
38_8c_50_68_d7_5c
24_fd_5b_4_1b_75
6c_56_97_35_39_f4
6c_72_20_c5_a_3f
70_2c_1f_3b_36_53
8_66_98_a2_21_9e
98_84_e3_e4_35_bd
b0_d5_9d_b9_f0_b4
84_c0_ef_2f_42_cc
88_de_a9_8_3_b9
b0_ce_18_27_9f_e4
b4_e6_2a_27_34_b7
bc_dd_c2_38_d_32
9c_8e_cd_a_33_1b
b0_fc_d_c9_0_4c
c0_97_27_81_67_99
d4_a3_3d_6b_1e_97
d8_f7_10_c3_34_e4
c0_97_27_73_aa_38
fc_a1_83_38_e0_2d
ec_fa_bc_2e_81_f7
d8_28_c9_10_b5_60
dc_4f_22_5b_1a_a3
f4_b8_5e_31_73_db


In [ ]:

start_time = time.time()
for directory in directories:
    target_dd = "/content/drive/MyDrive/TLSHandshakesTraffic/"+directory+"/" #target device directory
    files = [f for f in os.listdir(target_dd) if os.path.isfile(os.path.join(target_dd, f))]
    all_features = []
    csv_features = []
    ch_features = []
    sh_features = []
    client_hello_features = {}
    server_hello_features = {}
    csv_fstr = ''
    pcap_files = [f for f in files if f.lower().endswith('.pcap')]
    print(directory)

    for file in pcap_files:
        pcap_file = target_dd+file
        packets = PcapReader(pcap_file)

        for packet in packets:
          if TLS in packet:
              tls_layer = packet[TLS]
          elif Raw in packet:
            if packet[Raw].load.startswith(b'\x16\x03'):
              tls_layer = parse_tls_data(packet)
          if tls_layer.haslayer(TLS):
              if tls_layer.type == 22:
                if tls_layer.msg[0].show(dump=True).find('msgtype') >= 0:

                  if tls_layer.msg[0].msgtype == 1:  # ClientHello
                      ch_fstr=''
                      if tls_layer.msg[0].show(dump=True).find('msglen') >= 0:
                        csv_fstr=csv_fstr+';'+str(tls_layer.msg[0].msglen)
                        ch_fstr=ch_fstr+str(tls_layer.msg[0].msglen)
                      else:
                        ch_fstr=ch_fstr+';0'
                        csv_fstr=csv_fstr+';0'

                      if tls_layer.msg[0].show(dump=True).find('random_bytes') >= 0:
                        ch_fstr=ch_fstr+';'+str(bytearray(tls_layer.msg[0].random_bytes).hex())
                        csv_fstr=csv_fstr+';'+str(bytearray(tls_layer.msg[0].random_bytes).hex())
                      else:
                        ch_fstr=ch_fstr+';0'
                        csv_fstr=csv_fstr+';0'

                      if tls_layer.msg[0].show(dump=True).find('cipherslen') >= 0:
                        ch_fstr=ch_fstr+';'+str(tls_layer.msg[0].cipherslen)
                        csv_fstr=csv_fstr+';'+str(tls_layer.msg[0].cipherslen)
                      else:
                        ch_fstr=ch_fstr+';0'
                        csv_fstr=csv_fstr+';0'

                      if tls_layer.msg[0].show(dump=True).find('extlen') >= 0:
                        csv_fstr=csv_fstr+';'+str(tls_layer.msg[0].extlen)
                        ch_fstr=ch_fstr+';'+str(tls_layer.msg[0].extlen)
                      else:
                        ch_fstr=ch_fstr+';0'
                        csv_fstr=csv_fstr+';0'
                      sig_count = 0
                      sin_count = 0
                      if tls_layer.msg[0].show(dump=True).find('\ext') >= 0:
                        for ext in tls_layer.msg[0].ext:
                            if ext.type == 13:  # SignatureAlgorithms
                                csv_fstr=csv_fstr+';'+str(ext.sig_algs_len)
                                ch_fstr=ch_fstr+';'+str(ext.sig_algs_len)
                                sig_count = 1
                            elif ext.type == 0:
                                csv_fstr=csv_fstr+';'+str(ext.servernames)
                                ch_fstr=ch_fstr+';'+str(ext.servernames)
                                sin_count = 1
                      if sig_count == 0:
                        csv_fstr=csv_fstr+';0;0'
                      if sin_count == 0:
                        csv_fstr=csv_fstr+';0'
                        ch_fstr=ch_fstr+';0'

                      ch_features.append(ch_fstr)


                  elif tls_layer.msg[0].msgtype == 2:  # ServerHello
                      sh_fstr=''
                      if tls_layer.msg[0].show(dump=True).find('msglen') >= 0:
                        sh_fstr=sh_fstr+str(tls_layer.msg[0].msglen)
                      else:
                        sh_fstr=sh_fstr+'; '

                      if tls_layer.msg[0].show(dump=True).find('random_bytes') >= 0:
                        sh_fstr=sh_fstr+';'+str(bytearray(tls_layer.msg[0].random_bytes).hex())
                        csv_fstr=csv_fstr+';'+str(bytearray(tls_layer.msg[0].random_bytes).hex())
                      else:
                        sh_fstr=sh_fstr+'; '
                        csv_fstr=csv_fstr+'; '

                      if tls_layer.msg[0].show(dump=True).find('cipher') >= 0:
                        sh_fstr=sh_fstr+';'+str(tls_layer.msg[0].cipher)
                        csv_fstr=csv_fstr+';'+str(tls_layer.msg[0].cipher)
                      else:
                        sh_fstr=sh_fstr+'; '
                        csv_fstr=csv_fstr+'; '


                      sh_fstr=sh_fstr+';'+str(packet.dport)
                      csv_fstr=csv_fstr+';'+str(packet.dport)
                      pack = tls_layer.show(dump=True)
                      certificate = extract_certificate(pack)
                      if certificate == " ":
                        sh_fstr=sh_fstr+'; '
                        csv_fstr=csv_fstr+'; '
                      else:
                        csv_fstr=csv_fstr+';'+certificate
                        sh_fstr=sh_fstr+';'+certificate
                      sh_features.append(sh_fstr)
                      csv_features.append(csv_fstr)
                      csv_fstr=''





    np.savetxt("/content/drive/MyDrive/TLSHandshakesTraffic/"+directory+'/'+"Data-new.csv", csv_features, delimiter=";", newline='\n', fmt='%s')
    np.savetxt("/content/drive/MyDrive/TLSHandshakesTraffic/"+directory+'/'+"ClientHello-new.csv", ch_features, delimiter=";", newline='\n', fmt='%s')
    np.savetxt("/content/drive/MyDrive/TLSHandshakesTraffic/"+directory+'/'+"ServerHello-new.csv", sh_features, delimiter=";", newline='\n', fmt='%s')
    elapsed_time = time.time() - start_time
    print(f"Execution time: done {elapsed_time:.2f} seconds")

d8_28_c9_10_b5_60


Could not parse TLS data: 'Unknown hash function null'


KeyboardInterrupt: 